TODO

generic feature showcase showing genesis features
demonstrate robot viewer
video recording
point to next notebook for intro
point to other notebook for advanced uses

In [3]:
from pathlib import Path
import random 

import roboticstoolbox as rtb
import sympy

from manipylator import Robot
from manipylator.utils import render_robot_from_template

# ManiPyLator Basics
ManiPyLator wraps around two main robotics libraries, robotics-toolbox-python & Genesis and tries to provide an interface to both simulated and analytical robotics. Below is an example of loading an arbitrary URDF and a forward kinematics sanity check with both libraries:

In [2]:
with render_robot_from_template("robots/empiric") as robot_urdf:
    manny = Robot(robot_urdf)
print(manny.model)

ERobot: measured, 6 joints (RRRRRR), dynamics, geometry, collision
┌──────┬──────────────────┬───────┬──────────────────┬─────────────────────────────────────────────────────────────────────────────┐
│ link │       link       │ joint │      parent      │                             ETS: parent to link                             │
├──────┼──────────────────┼───────┼──────────────────┼─────────────────────────────────────────────────────────────────────────────┤
│    0 │ base             │       │ BASE             │ SE3()                                                                       │
│    1 │ carriage_1       │     0 │ base             │ SE3(-7.47e-06, -2.937e-06, 0.037; -180°, -7.914e-41°, -6.361e-15°) ⊕ Rz(q0) │
│    2 │ shoulder_lift    │     1 │ carriage_1       │ SE3(-0.065, -2.469e-06, -0.055; 180°, 90°, 180°) ⊕ Rz(q1)                   │
│    3 │ elbow            │     2 │ shoulder_lift    │ SE3(0.355, -2.338e-06, -4.032e-06; -180°, -3.45e-15°, 3.069e-12°) ⊕ Rz(q2)  │
│ 

For an arbitrary poes, we'll calculate the forward kinematics using both RTB and Genesis and compare:

In [3]:
arbitrary_pose = [random.uniform(-1, 1) for _ in range(6)]
print(arbitrary_pose)

[-0.4720275388975417, -0.18300879977360895, 0.6536733910306669, 0.3516591658552317, -0.7404998970643253, 0.7387880912391709]


In [4]:
forward_kinematics_rtb = manny.model.fkine(arbitrary_pose)
forward_kinematics_rtb

  -0.8065    0.4478    0.386    -0.2354    
   0.4828    0.1219    0.8672    0.2015    
   0.3413    0.8858   -0.3145    0.7165    
   0         0         0         1         


In [5]:
translation_rtb = forward_kinematics_rtb.t
translation_rtb

array([-0.23542257,  0.20153512,  0.71652318])

Both approaches should return True above, which means they calculate comparable solutios down to the 10 micron range

In [4]:

# Load robot from URDF
robot = manny.model

# Get the ETS
ets = robot.ets()

# Create symbolic variables for each joint
q_syms = sympy.symbols(f'q0:{robot.n}')

# Substitute symbolic variables into the ETS to get a symbolic SE3
T = ets.eval(q_syms)  # This returns an SE3 object with symbolic entries

# To get the underlying symbolic matrix:
# T_matrix = sympy.Matrix(T)

In [9]:
print("Shape:", T.shape)
print("First element:", T[0, 0])

Shape: (4, 4)
First element: 1.95980826913726e-64*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.08215137033126e-27*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.08215137033126e-27*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 1.95980826913726e-64*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 1.96589028693246e-42*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 2.87158583120317e-33*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 2.87158583120317e-33*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-

In [12]:
print(sympy.latex(T))  # LaTeX string

\mathtt{\text{[[1.95980826913726e-64*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.08215137033126e-27*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.08215137033126e-27*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 1.95980826913726e-64*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 1.96589028693246e-42*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 2.87158583120317e-33*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 2.87158583120317e-33*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.6732051

In [15]:
T_matrix = sympy.Matrix(T)

In [18]:
subs_dict = {q: 0 for q in q_syms}
T_numeric = T_matrix.subs(subs_dict).evalf(n=3)
T_numeric

Matrix([
[-8.98e-6, -3.67e-6,     1.0, -0.129],
[     1.0, -2.65e-6, 8.98e-6, 1.6e-6],
[ 2.65e-6,      1.0, 3.67e-6,  0.811],
[       0,        0,       0,    1.0]])

# Where To Now?
The above demonstrates the two available interfaces in ManiPylator. Based on your interests you might continue to explore our visual robotics demonstration or maybe try to derive a symbolic result. For whatever path you're interested in, check out our Robotics resources page.

For beginners, we recommend to:


For experienced robotocists, check out our

Point to RTB and 